In [ ]:
import warnings
import json
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
with open(r"global_config.json","r") as fb:
    global_config = json.load(fb)


In [ ]:
#Initializing the llm
llm = AzureChatOpenAI(
    deployment_name=global_config["model_name"],
    openai_api_key=global_config["openai_api_key"],
    azure_endpoint=global_config["openai_api_base"],
    openai_api_version=global_config["openai_api_version"],
    temperature=0
)

In [ ]:
user_conversations = {}  # Store user conversation history
user_topics = {}  # Store user preferred genres

genres = ["Sci-Fi", "Horror", "Thriller", "Action", "Drama", "Comedy", "Fantasy"]

def update_user_history(user_id, message):
    """Stores the user's messages to maintain conversation history."""
    if user_id not in user_conversations:
        user_conversations[user_id] = []
    user_conversations[user_id].append(message)

def determine_preferred_genres(user_id, message):
    """ Determines preferred genres from user conversation. """
    if user_id not in user_topics:
        user_topics[user_id] = set()
    
    detected_genres = {genre for genre in genres if genre.lower() in message.lower()}
    if detected_genres:
        user_topics[user_id].update(detected_genres)
    
    return user_topics[user_id]

In [ ]:
prompt = PromptTemplate(
    input_variables=["history", "message", "genres"],
    template="""
    You are a helpful chatbot that discusses movies. 
    The user has been talking about: {history}. 
    The genres they have discussed are: {genres}.
    
    If the user asks for recommendations, suggest popular movies from the discussed genres.
    Otherwise, continue engaging with them in a meaningful way.
    
    Respond to: '{message}'
    """
)

chat_chain = prompt | llm  

def get_chatbot_response(user_id, message):
    """Generates a chatbot response based on conversation history and user preferences."""
    update_user_history(user_id, message)
    preferred_genres = determine_preferred_genres(user_id, message)
    history = " | ".join(user_conversations[user_id][-5:])  

    response = chat_chain.invoke({"history": history, "message": message, "genres": list(preferred_genres)})
    return response.content  


In [ ]:
# Create input box and button
input_box = widgets.Text(placeholder="Type your message...")
send_button = widgets.Button(description="Send")
output_area = widgets.Output()

user_id = "user_123"

# Display chatbot's first message
with output_area:
    print("Chatbot: Hi! Let's chat about movies! Ask me about any genre.")

In [ ]:
def on_send_message(change):
    """Handles user input, updates the UI, and manages chatbot interaction."""
    user_input = input_box.value.strip()
    input_box.value = ""  # Clear input field
    
    if user_input.lower() == "exit":
        with output_area:
            print("Chatbot: Goodbye! Have a great day!")
        send_button.disabled = True  # Disable further input
        return
    
    response = get_chatbot_response(user_id, user_input)
    
    # Display conversation
    with output_area:
        print(f"🧑‍💻YOU: {user_input}")
        print(f"🤖CHATBOT: {response}")
        print("-----------------------------------------------")


In [ ]:
# Attach event listener to the button
send_button.on_click(on_send_message)

# Display UI elements
display(output_area, input_box, send_button)